In [2]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm

In [3]:

# Generate synthetic data
def generate_data(n_samples, n_features, true_theta, true_thresholds, sigma=1.0):
    """
    Generate synthetic data for ordinal regression using a latent variable model.

    Parameters:
    n_samples : int
        Number of samples to generate.
    n_features : int
        Number of features.
    true_theta : ndarray
        True parameter vector for features.
    true_thresholds : ndarray
        True thresholds for ordinal categories.
    sigma : float
        Standard deviation of the latent variable.

    Returns:
    X : ndarray
        Feature matrix.
    y : ndarray
        Ordinal target vector.
    """
    X = np.random.randn(n_samples, n_features)  # Random feature matrix
    z = X @ true_theta + np.random.normal(0, sigma, size=n_samples)  # Latent variable
    thresholds = np.concatenate([[-np.inf], true_thresholds, [np.inf]])  # Include boundaries

    # Determine y based on thresholds
    y = np.digitize(z, thresholds) - 1  # Subtract 1 to make y start from 0

    return X, y

In [26]:
# True parameters for synthetic data
n_samples = 1000
n_features = 3
n_categories = 4  # K=4 ordinal categories
true_theta = np.array([1.0, -1.0, 0.5])  # True feature weights
true_thresholds = np.array([-5.0, 0.35, 1.0])  # True thresholds
sigma_true = 1.0  # True sigma

# Generate synthetic data
X, y = generate_data(n_samples, n_features, true_theta, true_thresholds, sigma=sigma_true)

In [27]:
X

array([[-1.91929931, -0.69315003,  0.38316362],
       [-0.19971782, -0.74760958,  1.30834944],
       [-0.05138498, -0.35874265, -0.4470689 ],
       ...,
       [ 0.9987484 , -1.62450618,  0.52863981],
       [ 1.94642874, -1.33544859, -0.05489007],
       [-0.63756448, -2.62269038, -0.7387322 ]])

In [28]:
y

array([1, 2, 1, 2, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 2, 3, 3, 1,
       1, 2, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 3, 1, 2, 2, 1, 3, 1, 3, 1, 1,
       1, 3, 1, 1, 1, 3, 3, 1, 1, 3, 1, 1, 3, 3, 3, 1, 1, 1, 3, 1, 1, 1,
       1, 1, 2, 2, 1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 3, 3, 2, 3, 1, 3, 1, 1,
       1, 1, 1, 1, 1, 3, 1, 1, 2, 2, 3, 1, 3, 1, 1, 3, 1, 2, 1, 1, 1, 3,
       2, 3, 3, 2, 3, 3, 2, 3, 1, 1, 1, 0, 3, 2, 2, 1, 1, 3, 1, 2, 3, 2,
       1, 1, 1, 1, 1, 3, 3, 2, 1, 1, 1, 3, 3, 1, 3, 2, 2, 1, 3, 1, 1, 3,
       3, 2, 1, 3, 1, 1, 2, 1, 1, 1, 3, 1, 3, 1, 3, 3, 3, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 3, 1, 2, 3, 1, 2, 3, 3, 1, 3, 3, 1, 1, 1, 2, 1,
       3, 1, 2, 1, 1, 1, 1, 1, 2, 3, 1, 1, 3, 2, 1, 1, 1, 1, 2, 2, 3, 1,
       1, 1, 3, 3, 1, 1, 3, 1, 1, 1, 3, 1, 3, 1, 1, 1, 3, 3, 2, 1, 3, 1,
       3, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 3,
       1, 1, 1, 3, 2, 1, 1, 3, 3, 1, 3, 1, 2, 2, 1, 1, 1, 1, 3, 1, 3, 3,
       1, 2, 3, 2, 1, 1, 1, 2, 3, 1, 3, 1, 1, 3, 1,

In [29]:
def ordinal_nll(X, y, theta, thresholds, sigma=1.0):
    """
    Compute the negative log-likelihood for ordinal regression (vectorized).

    Parameters:
    X : ndarray
        Feature matrix where each row represents a sample and each column represents a feature.
    y : ndarray
        Target vector where each element is the target ordinal value for the corresponding sample.
    theta : ndarray
        Parameter vector for features (weights).
    thresholds : ndarray
        Thresholds for ordinal categories (K-1 cutpoints).
    sigma : float
        Standard deviation of the latent variable.

    Returns:
    float
        The negative log-likelihood value.
    """
    # Compute the linear predictor
    linear_pred = X @ theta  # shape: [n_samples]

    # Prepend -inf and append +inf to thresholds for boundary conditions
    thresholds = np.concatenate([[-np.inf], thresholds, [np.inf]])
    
    # Compute probabilities for each category (vectorized)
    cdf_upper = norm.cdf((thresholds[y + 1] - linear_pred) / sigma)
    cdf_lower = norm.cdf((thresholds[y] - linear_pred) / sigma)
    prob_y = cdf_upper - cdf_lower

    # Avoid log(0) with numerical stability
    prob_y = np.clip(prob_y, 1e-15, 1 - 1e-15)
    
    # Compute negative log-likelihood
    return -np.sum(np.log(prob_y))

In [30]:

def unpack_params(params, n_features):
    """Helper function to unpack weights and thresholds."""
    theta = params[:n_features]
    thresholds = params[n_features:]
    return theta, thresholds

# Wrapper for optimization function
def nll_wrapper(params, X, y, sigma):
    theta, thresholds = unpack_params(params, X.shape[1])
    return ordinal_nll(X, y, theta, thresholds, sigma=sigma)



In [32]:
# Initial parameters for optimization
theta_init = np.random.randn(n_features)  # Random initial weights
thresholds_init = np.linspace(-5, 5, n_categories - 1)  # Evenly spaced initial thresholds
params_init = np.concatenate([theta_init, thresholds_init])  # Combine parameters

# Optimize using scipy's minimize
sigma_fixed = sigma_true  # Fix sigma to true value
result = minimize(
    lambda params: nll_wrapper(params, X, y, sigma_fixed),
    params_init,
    method='BFGS'
)

# Extract results
theta_opt, thresholds_opt = unpack_params(result.x, n_features)

# Print results
print("Optimization Result:")
print("Success:", result.success)
print("Message:", result.message)
print("True Parameters (theta):", true_theta)
print("Estimated Parameters (theta):", theta_opt)
print("True Thresholds:", true_thresholds)
print("Estimated Thresholds:", thresholds_opt)
print("Function Value (Negative Log-Likelihood):", result.fun)

Optimization Result:
Success: True
Message: Optimization terminated successfully.
True Parameters (theta): [ 1.  -1.   0.5]
Estimated Parameters (theta): [ 0.37310853  1.0823132  -0.51291999]
True Thresholds: [-5.    0.35  1.  ]
Estimated Thresholds: [-5.  0.  5.]
Function Value (Negative Log-Likelihood): 8662.076


# `m01` Ordinal Bayesian Modeling with Fixed Cutpoints

In [33]:
import numpy as np
import numpyro
import numpyro.distributions as dist
from numpyro.infer import MCMC, NUTS
import jax
import jax.numpy as jnp

In [34]:
fixed_thresholds = np.array([-1.0, 0, 1])  # Fixed cutpoints

def ordinal_probit_model(X, y, alpha_cutpoints, sigma_beta=2.0, sigma_z=1.0):
    """
    Bayesian Ordinal Probit Model with fixed cutpoints and latent variables.
    For identification, we fix sigma_z=1.0 (standard probit model assumption).
    
    Args:
        X (array): Input features of shape (n_samples, n_features)
        y (array): Target ordinal labels of shape (n_samples,)
        alpha_cutpoints (array-like): Fixed cutoff points for ordinal categories
        sigma_beta (float): Standard deviation for beta prior
    """
    # Get dimensions
    n_samples, num_features = X.shape
    num_categories = len(alpha_cutpoints) + 1
        
    # Sample regression coefficients from prior with larger variance
    beta = numpyro.sample(
        "beta",
        dist.Normal(
            loc=jnp.zeros(num_features),
            scale=jnp.ones(num_features) * sigma_beta
        )
    )
    
    z = jnp.dot(X, beta)
    
    # Compute probabilities using the standard normal CDF
    probs = jnp.zeros((n_samples, num_categories))
    
    # Binary classification case
    # P(y=1) = P(z ≤ α)
    p1 = numpyro.distributions.Normal(0, 1).cdf((alpha_cutpoints[0] - z))
    probs = probs.at[:, 0].set(p1)
    probs = probs.at[:, 1].set(1 - p1)

    # Ensure probabilities sum to 1 and are positive
    probs = jnp.clip(probs, 1e-8, 1.0)
    probs = probs / probs.sum(axis=1, keepdims=True)
    
    # Sample observations
    with numpyro.plate("obs", n_samples):
        numpyro.sample("y", dist.Categorical(probs=probs), obs=y)



In [35]:
nuts_kernel = NUTS(ordinal_probit_model)
mcmc = MCMC(nuts_kernel, num_warmup=1500, num_samples=2000, num_chains=1)
mcmc.run(jax.random.PRNGKey(0), X=X, y=y, alpha_cutpoints=fixed_thresholds)


sample: 100%|██████████| 3500/3500 [00:01<00:00, 1851.99it/s, 5 steps of size 7.80e-01. acc. prob=0.91] 


In [36]:
mcmc.print_summary()


                mean       std    median      5.0%     95.0%     n_eff     r_hat
   beta[0]     -0.46      0.09     -0.47     -0.61     -0.31   1733.17      1.00
   beta[1]      0.48      0.09      0.48      0.33      0.63   1561.80      1.00
   beta[2]     -0.20      0.08     -0.20     -0.33     -0.08   1594.31      1.00

Number of divergences: 0


In [37]:
import numpyro
import numpyro.distributions as dist
from jax import random
import jax.numpy as jnp
from numpyro.infer import MCMC, NUTS

import jax.numpy as jnp
from jax.scipy.stats import norm
from numpyro.distributions import constraints, CategoricalProbs
from numpyro.distributions.util import promote_shapes

class OrderedProbit(CategoricalProbs):
    """
    A categorical distribution with ordered outcomes, using a Probit link.

    :param numpy.ndarray predictor: predictions in the real domain; typically the output
        of a linear model.
    :param numpy.ndarray cutpoints: positions in the real domain to separate categories.
    """

    arg_constraints = {
        "predictor": constraints.real,
        "cutpoints": constraints.ordered_vector,
    }

    def __init__(self, predictor, cutpoints, *, validate_args=None):
        if jnp.ndim(predictor) == 0:
            (predictor,) = promote_shapes(predictor, shape=(1,))
        else:
            predictor = predictor[..., None]
        predictor, cutpoints = promote_shapes(predictor, cutpoints)
        self.predictor = predictor[..., 0]
        self.cutpoints = cutpoints

        # Compute cumulative probabilities using the probit link (normal CDF)
        cdf = norm.cdf
        probs = jnp.concatenate([
            cdf(self.cutpoints[..., 0] - self.predictor[..., None]),
            cdf(self.cutpoints[..., 1:] - self.predictor[..., None]) -
            cdf(self.cutpoints[..., :-1] - self.predictor[..., None]),
            1.0 - cdf(self.cutpoints[..., -1] - self.predictor[..., None])
        ], axis=-1)

        super(OrderedProbit, self).__init__(probs, validate_args=validate_args)

    @staticmethod
    def infer_shapes(predictor, cutpoints):
        batch_shape = jnp.broadcast_shapes(predictor.shape, cutpoints[:-1].shape)
        event_shape = ()
        return batch_shape, event_shape

    def entropy(self):
        raise NotImplementedError


# Step 1: Define the model in NumPyro
def ordinal_regression_model(X, y=None, n_categories=4):
    n_features = X.shape[1]
    
    # Priors for regression coefficients
    beta = numpyro.sample("beta", dist.Normal(jnp.zeros(n_features), jnp.ones(n_features)))
    
    # Dirichlet prior for category probabilities
    alpha = jnp.ones(n_categories)
    p = numpyro.sample("p", dist.Dirichlet(alpha))
    
    # Cumulative probabilities derived from p
    q = jnp.cumsum(p[:-1])
    
    # Cut points derived using probit link (inverse CDF of normal distribution)
    cut_points = numpyro.deterministic("cut_points", dist.Normal(0, 1).icdf(q))
    
    # Linear predictor for latent variable z
    z = jnp.dot(X, beta)
    

    # Observational model
    with numpyro.plate("data", X.shape[0]):
        numpyro.sample("y", OrderedProbit(predictor=z, cutpoints=cut_points), obs=y)

        
# Step 2: Generate synthetic data
cut_points_true = true_thresholds
# Step 3: Run MCMC with NumPyro
rng_key = random.PRNGKey(0)
kernel = NUTS(ordinal_regression_model)
mcmc = MCMC(kernel, num_warmup=1000, num_samples=2000, num_chains=4)
mcmc.run(rng_key, X=jnp.array(X), y=jnp.array(y), n_categories=4)

# Step 4: Extract results
posterior_samples = mcmc.get_samples()

# Display posterior summaries
import arviz as az
idata = az.from_numpyro(mcmc)
az.summary(idata, var_names=["beta", "cut_points"])


/var/folders/1h/y7css5mj58d0d43d5rmmv58m0000gp/T/ipykernel_75333/3497928026.py:87: UserWarning: There are not enough devices to run parallel chains: expected 4 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(4)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  mcmc = MCMC(kernel, num_warmup=1000, num_samples=2000, num_chains=4)
sample: 100%|██████████| 3000/3000 [00:05<00:00, 524.85it/s, 3 steps of size 3.02e-01. acc. prob=0.85] 


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta[0],0.938,0.047,0.847,1.020,0.001,0.001,2127.0,3111.0,1.0
beta[1],-1.004,0.048,-1.094,-0.913,0.001,0.001,2132.0,2431.0,1.0
beta[2],0.441,0.047,0.354,0.528,0.001,0.001,3561.0,3996.0,1.0
cut_points[0],-4.519,0.254,-4.975,-4.039,0.005,0.004,2435.0,2185.0,1.0
cut_points[1],0.372,0.052,0.279,0.472,0.001,0.001,2744.0,2692.0,1.0
cut_points[2],0.950,0.053,0.849,1.051,0.001,0.001,3015.0,3408.0,1.0


In [21]:
cut_points_true

array([-1.,  0.,  1.])